# Analysing hotel bookings, demand and factors affecting cancelations.

**Team Members:**
Zizheng Gao,
Menghe Liu (Crystal),
Eunjin Jeong (Jenny),
Qianru Ai,
Ta-Wei Wang (David),
Aash Gohil

## Project Proposal:
- To analyse existing hotel bookings, intrinsic difference's between the hotels, understanding the demographic data, and also the sources of demand.

- Additionally, analyse and visualise relationships between the independent variables (like demographic data, lead time, change in booking, deposit paid, booking type etc.) and the probability of cancelation.
 
- If possible, To try and predict using multivariate logistic regression or other ML Models, the probability of cancellation and predict demand, using ARIMA, or other forecasting models.

**Data Source:** “Hotel Booking Demand Dataset”, Data in Brief Journal Vol 22, Feb 2019 (https://www.kaggle.com/jessemostipak/hotel-booking-demand)

## About the Data

This data describes hotel demand data for 2 hotels based in Portugal. One of the hotels is a resort hotel and the other is a city hotel. The datasets cointains 32 variables and 119k rows. Each observation/row represents a hotel booking, the data ranges from between the 1st of July of 2015 and the 31st of August 2017

In [5]:
%%bigquery
Select * from `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset` LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.12rows/s]


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,Arrival_month_num
0,City Hotel,1,26,2017,January,1,1,1,0,1,...,9,NULL,0,Transient,128.0,0,2,Canceled,2016-12-17,1
1,City Hotel,1,26,2017,January,1,1,2,2,2,...,9,NULL,0,Transient,103.7,0,0,Canceled,2016-12-17,1
2,Resort Hotel,1,42,2017,January,1,2,1,0,2,...,242,NULL,0,Transient,42.0,0,0,Canceled,2016-12-17,1
3,Resort Hotel,1,42,2017,January,1,2,1,0,2,...,242,NULL,0,Transient,42.0,0,0,Canceled,2016-12-17,1
4,City Hotel,1,43,2017,January,1,5,1,3,2,...,9,NULL,0,Transient,74.8,0,0,Canceled,2016-12-19,1


## 1. Demographic Analysis

### 1.1 Which country do most vistors come from?

Majoritiy of the vistors come from Portugal, which counts for 40.7% of the total visitors. This makes sense since both the hotels are based in Portugal.

In [7]:
%%bigquery
select country, count(*) as booing_num, round(count(*)*100/(select count(*) from `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`),2)
as percentage_of_total from `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
group by country
order by percentage_of_total desc
limit 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.81rows/s]


,country,booing_num,percentage_of_total
0,PRT,48590,40.70
1,GBR,12129,10.16
2,FRA,10415,8.72
3,ESP,8568,7.18
4,DEU,7287,6.10


### 1.2 What is the most common vistors' group size

68% of the bookings are made by a group of two adults, and more than 90% of the bookings are made by groups without kids.

In [9]:
%%bigquery
SELECT distinct concat(adults,' adults, ',children+babies,' children') as type, count(hotel) as num,
round(count(hotel)*100/(select count(*) from `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`),2) as percentage_of_total
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
group by type
order by num DESC
limit 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.26rows/s]


,type,num,percentage_of_total
0,"2 adults, 0 children",81560,68.31
1,"1 adults, 0 children",22577,18.91
2,"3 adults, 0 children",5666,4.75
3,"2 adults, 1 children",4658,3.90
4,"2 adults, 2 children",3369,2.82


### 1.3 Bookings made by First time customers vs Repeat customers

Most of the bookings are made by first-time guests, and only 3.19% of the bookings are made by repeated guests.

In [11]:
%%bigquery
SELECT CASE 
    when is_repeated_guest = 1 then 'Repeated'
    Else 'New_Guest'
    End as repeated
,count(*) as guest_num, round(count(*)*100/(select count(*) from `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`),2)
as percentage_of_total FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset` 
group by repeated

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.26rows/s]


,repeated,guest_num,percentage_of_total
0,New_Guest,115580,96.81
1,Repeated,3810,3.19


## 2. Hotel Analysis

### 2.1 Average wait time for each hotel and its impact

The average waiting days to get a room are longer for city hotels than resort hotels. The average waiting days for get a room in city hotels is about 3 days, compared to 0.5 days for resort hotels.

This can impact cancellation, with
City hotel bookings being more likely to be cancelled, since 41.73% of the times a reservation of a city hotel is canceled, and only 27.76% of the times a resort hotel is canceled by guests.

In [14]:
%%bigquery
SELECT hotel, AVG(days_in_waiting_list) AS avg_waiting_days,
ROUND(sum(is_canceled)*100/count(is_canceled),2) AS cancellation_rate
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
GROUP BY hotel

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.30rows/s]


,hotel,avg_waiting_days,cancellation_rate
0,City Hotel,3.226774,41.73
1,Resort Hotel,0.527758,27.76


### 2.2 Nights spent for each hotel and average daily spend

On average, guest spend close to a day longer at the resort hotel than the city hotel. However Guests spend more per day during their stay at the city hotel than at resport hotel by 15 euro

In [55]:
%%bigquery
select hotel, round(avg(stays_in_weekend_nights),2) Avg_weekend_nights
, round(avg(stays_in_week_nights),2) Avg__weekday_nights
, round(avg(stays_in_week_nights + stays_in_weekend_nights),2) Avg_ttl_nights
, round(avg(adr),2) Avg_Daily_spend_during_stay
from `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset` 
where is_canceled != 1 group by 1

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.34rows/s]


,hotel,Avg_weekend_nights,Avg__weekday_nights,Avg_ttl_nights,Avg_Daily_spend_during_stay
0,Resort Hotel,1.13,3.01,4.14,90.79
1,City Hotel,0.80,2.12,2.92,105.75


### 2.3 Which type of hotel is more popular among guests with children and babies?

For families with kids and babies, resort hotels are more popular than city hotels.

Although there are more guests with kids stayed in city hotels, this type of guest actually prefers staying in resort hotels. Because the total number of reservations is higher for city hotels than resort hotels, the absolute percentage for families with kids staying in resort hotel among total reservations in resort hotels is higher than that in city hotels. Moreover, among all families with kids or babies, 59.03% of them chose to stay in resort hotels, and 40.97% of them stayed in city hotels. In conclusion, resort hotels are prefered by families with kids and babies.

In [15]:
%%bigquery
with main as 
(SELECT *, round(Guest_with_kids*100/Total_Bookings,2) as Absolute_pop_percent
FROM (
SELECT hotel, Count(if(children != 0 or babies !=0, True, Null)) as Guest_with_kids, count(*) AS Total_Bookings
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset` group by 1
))

select *, round(Absolute_pop_percent*100/(select sum(Absolute_pop_percent) from main),2) as Relative_Popularity_for_guest_w_kids
from main;

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.27rows/s]


,hotel,Guest_with_kids,Total_Bookings,Absolute_pop_percent,Relative_Popularity_for_guest_w_kids
0,City Hotel,5403,79330,6.81,40.97
1,Resort Hotel,3929,40060,9.81,59.03


### 2.4 Which type of hotels is the more popular among guests without children?

For guests without children, city hotels are more popular than resort hotels, but the difference in popularity in this case is not significant.

Among all the reservations in city hotels, 93.19% of guests stayed in the hotel without kids or babies. Among all the reservations in resort hotels, 90.19% of guests stayed in the hotel without kids or babies. Moreover, among all guests stayed in hotels without kids, 50.82% of them chose to stay in city hotels, and 49.18% of them stayed in resort hotels.

In [17]:
%%bigquery
with main2 as
(SELECT *, round(Guest_without_kids*100/Total_Bookings,2) as Absolute_pop_percent_no_kids
FROM (
SELECT hotel, Count(if(children = 0 and babies =0, True, Null)) as Guest_without_kids, count(*) AS Total_Bookings
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset` group by 1
))
select *, round(Absolute_pop_percent_no_kids*100/(select sum(Absolute_pop_percent_no_kids) from main2),2)as Relative_Popularity_for_guest_no_kids
from main2;

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.37rows/s]


,hotel,Guest_without_kids,Total_Bookings,Absolute_pop_percent_no_kids,Relative_Popularity_for_guest_no_kids
0,City Hotel,73927,79330,93.19,50.82
1,Resort Hotel,36131,40060,90.19,49.18


## 3. Demand Analysis

### 3.1 What type of booking channel do guests prefer the most?

The result shows that ONLINE TA (Online Travel Agent) is the most preferred booking channel by guests accounting for close to 47% of the bookings, followed by Offline TA/TO (Offline Travel Agent/ Tour Operator) accounting for roughly 20% of bookings

In [34]:
%%bigquery
SELECT market_segment, count(*) as num_guests, round(count(*)*100/(select count(*) from `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`),2) 
AS percent_of_guests
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
GROUP BY market_segment
ORDER BY 2 DESC

Downloading: 100%|██████████| 8/8 [00:01<00:00,  4.91rows/s]


,market_segment,num_guests,percent_of_guests
0,Online TA,56477,47.30
1,Offline TA/TO,24219,20.29
2,Groups,19811,16.59
3,Direct,12606,10.56
4,Corporate,5295,4.44
5,Complementary,743,0.62
6,Aviation,237,0.20
7,Undefined,2,0.00


### 3.2 What type of meal plans do guests generally go for?

The result reports that guests generally go for breakfast-only plan, with 77 percent opting for it.

In [35]:
%%bigquery
SELECT case when meal = 'BB' then 'Only Breakfast'
when meal = 'HB' then 'Breakfast + Additional Meal'
when meal = 'FB' then 'Breakfast, Lunch and Dinner'
when meal = 'SC' or meal = 'Undefined' then 'No meals' end as Meal_type
, count(*) AS num_guest
,round(count(*)*100/(select count(*) from `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`),2) 
AS percent_of_guests
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
GROUP BY meal_type
ORDER BY num_guest DESC

#Undefined/SC – no meal package;
#BB – Bed & Breakfast;
#HB – Half board (breakfast and one other meal – usually dinner);
#FB – Full board (breakfast, lunch and dinner)

Downloading: 100%|██████████| 4/4 [00:01<00:00,  2.75rows/s]


,Meal_type,num_guest,percent_of_guests
0,Only Breakfast,92310,77.32
1,Breakfast + Additional Meal,14463,12.11
2,No meals,11819,9.90
3,"Breakfast, Lunch and Dinner",798,0.67


### 3.3 Which months have the most reservations? and what is its average booking lead time?

On average there are more bookings in August, July and these months also correspond with a higher boooking lead time.

In [36]:
%%bigquery
SELECT arrival_date_month, count(*) AS num_bookings, round(avg(lead_time),2) lead_time
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
GROUP BY arrival_date_month
ORDER BY num_guest DESC

Downloading: 100%|██████████| 12/12 [00:01<00:00,  8.92rows/s]


,arrival_date_month,num_guest,lead_time
0,August,13877,121.14
1,July,12661,136.32
2,May,11791,117.65
3,October,11160,122.99
4,April,11089,94.82
5,June,10939,128.25
6,September,10508,136.68
7,March,9794,70.12
8,February,8068,47.86
9,November,6794,76.58


### 3.4 Which month do guests spend the longest time in a hotel?

On average, Guest have longer stays in July and August spending around 2.25 and 2.12 days, respectively.

In [39]:
%%bigquery
SELECT arrival_date_month, round(avg(stays_in_weekend_nights + stays_in_weekend_nights),2) as avg_nights_per_stay
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
GROUP BY 1 order by 2 desc

Downloading: 100%|██████████| 12/12 [00:01<00:00,  8.47rows/s]


,arrival_date_month,avg_nights_per_stay
0,July,2.24
1,August,2.12
2,April,1.90
3,September,1.88
4,June,1.84
5,October,1.79
6,May,1.74
7,December,1.73
8,March,1.72
9,February,1.71


### 3.5 When do hotels have the most reservations by date?

From the result the maximum number of bookings were for 5th Dec, 2015, which coresponded with the holiday "Zweiter Advent" in Portugal.
Even though on average, December has one of the lowest number of bookings. 

In [40]:
%%bigquery
SELECT PARSE_DATE("%Y/%B/%d", concat(arrival_date_year,"/",arrival_date_month,"/",arrival_date_day_of_month)) date,
count(*) as reservation_num
from `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
group by 1
order by 2 desc;

Downloading: 100%|██████████| 793/793 [00:01<00:00, 583.14rows/s]


,date,reservation_num
0,2015-12-05,448
1,2016-11-07,366
2,2015-10-16,356
3,2016-10-13,344
4,2015-09-18,340
...,...,...
788,2015-12-15,29
789,2015-11-15,28
790,2015-12-07,27
791,2015-11-29,20


## 4. Cancellation Analysis

### 4.1 Impact of lead days on cancelation

The result shows that on average, the earlier a guest make's a booking the higher the cancellation rate is, with the cancellation rates of 600-649 and 500-600 lead days being highest, of 100%, and cancellation rate of lead time = 0 is the lowest, of 6.78%.

In [43]:
%%bigquery
SELECT
CASE
when lead_time=0 then '0'
when lead_time>=1 and lead_time<50 then '1-49'
when lead_time>=50 and lead_time <100 then '50-99'
when lead_time>=100 and lead_time <150 then '100-149'
when lead_time>=150 and lead_time <200 then '150-199'
when lead_time>=200 and lead_time <250 then '200-249'
when lead_time>=250 and lead_time <300 then '250-299'
when lead_time>=300 and lead_time <350 then '300-349'
when lead_time>=350 and lead_time <400 then '350-399'
when lead_time>=400 and lead_time <450 then '400-449'
when lead_time>=450 and lead_time <500 then '450-499'
when lead_time>=500 and lead_time <550 then '500-549'
when lead_time>=550 and lead_time <600 then '550-600'
when lead_time>=600 and lead_time <650 then '600-649'
when lead_time>=650 and lead_time <700 then '650-699'
when lead_time>=700 then '>=700'
end AS lead_time_range, count(*) AS total_frequency,
SUM(is_canceled) AS cancellation_frequency, round(SUM(is_canceled)*100/count(*),2) AS cancellation_rate
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
WHERE lead_time IS NOT null
GROUP BY lead_time_range
ORDER BY 4 DESC;

Downloading: 100%|██████████| 15/15 [00:01<00:00, 10.87rows/s]


,lead_time_range,total_frequency,cancellation_frequency,cancellation_rate
0,550-600,119,119,100.00
1,600-649,145,145,100.00
2,500-549,215,148,68.84
3,450-499,636,437,68.71
4,300-349,4056,2772,68.34
5,350-399,1809,1200,66.33
6,400-449,998,659,66.03
7,250-299,5692,3398,59.70
8,200-249,7118,3408,47.88
9,150-199,12085,5558,45.99


We divided the lead time according to several bars, and calculated the cancellation rate for each

### 4.2 Are guests who make more changes less likely or more likely to cancel？

Usually, when a guest makes change, it would increase the cost of the hotel. That is why we wanted to investigate the relationship between changes the guests made and the cancellation rate, because if the guest who made several changes finally cancel his/her order, it would cause more loss of the hotel.

From the results, we can see that guests who make changes in their bookings have a lower cancellation rate than those who do not.

In [47]:
%%bigquery
SELECT case when booking_changes = 0 then 'No' else 'Yes' end as change_in_booking
, COUNT(*) AS total_frequency, SUM(is_canceled) AS cancellation_frequency, 
round(SUM(is_canceled)*100/count(is_canceled),2) AS cancellation_rate
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
GROUP BY change_in_booking
ORDER BY cancellation_rate DESC

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.27rows/s]


,change_in_booking,total_frequency,cancellation_frequency,cancellation_rate
0,No,101314,41391,40.85
1,Yes,18076,2833,15.67


### 4.3 Cancellation rate by different hotel

From the result, can be concluded that City Hotel experienced a higher cancellation rate.

In [50]:
%%bigquery
SELECT hotel, COUNT(*) AS total_frequency, SUM(is_canceled) AS cancellation_frequency,
round(SUM(is_canceled)*100/count(is_canceled),2) AS cancellation_rate
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
GROUP BY hotel
ORDER BY hotel;

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.45rows/s]


,hotel,total_frequency,cancellation_frequency,cancellation_rate
0,City Hotel,79330,33102,41.73
1,Resort Hotel,40060,11122,27.76


### 4.4 Cancellation rate by distribution channel

Distribution channels are important for sales. Therefore, investigating the relationship between distribution channel and cancellation rate is necessary.

It can be seen that orders of Direct have the lowest cancellation rate, while orders from TA/TO (Travel Agent/Tour Opeartor) and other channels have the highest cancellation rate. Thus, the hotels should focus on building there own distribution channel.

In [51]:
%%bigquery
SELECT distribution_channel, count(*) AS total_frequency, SUM(is_canceled) AS cancellation_frequency, 
round(SUM(is_canceled)*100/COUNT(*),2) AS cancellation_rate
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
GROUP BY distribution_channel
ORDER BY cancellation_rate

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.31rows/s]


,distribution_channel,total_frequency,cancellation_frequency,cancellation_rate
0,Direct,14645,2557,17.46
1,GDS,193,37,19.17
2,Corporate,6677,1474,22.08
3,TA/TO,97870,40152,41.03
4,Undefined,5,4,80.00


### 4.5 Cancellation rate by deposit type

The deposit is a means for the hotel to reduce the loss caused by the cancellation of the customer. But does it increase the probability of cancellation to some extent?

We can see that the harshest, non-refundable deposit type actually has the highest cancellation rate. Meanwhile, refundable deposits have the lowest cancellation rates.</br>
We also speculate that the non-refundable deposit type may have reduced the desire of customers to book this hotel. So all things considered, setting a non-refundable deposit is not a good way to reduce losses.

In [52]:
%%bigquery
SELECT deposit_type, count(*) AS total_frequency, SUM(is_canceled) AS cancellation_frequency, 
round(SUM(is_canceled)*100/COUNT(*),2) AS cancellation_rate
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
GROUP BY deposit_type

Downloading: 100%|██████████| 3/3 [00:01<00:00,  1.87rows/s]


,deposit_type,total_frequency,cancellation_frequency,cancellation_rate
0,No Deposit,104641,29694,28.38
1,Non Refund,14587,14494,99.36
2,Refundable,162,36,22.22


### 4.6 Canceltation rate by customer type

It can be concluded that the Transient customers have the highest cancellation_rate, while the Group customers have the lowest cancellation rates.

In [54]:
%%bigquery
SELECT customer_type, count(*) AS total_frequency, SUM(is_canceled) AS cancellation_frequency, 
round(SUM(is_canceled)*100/COUNT(*),2) AS cancellation_rate
FROM `ba775-project-t-2-b-1.hotel_booking.hotel_booking_dataset`
GROUP BY customer_type
ORDER BY SUM(is_canceled)/COUNT(*) DESC, SUM(is_canceled) DESC

Downloading: 100%|██████████| 4/4 [00:01<00:00,  2.90rows/s]


,customer_type,total_frequency,cancellation_frequency,cancellation_rate
0,Transient,89613,36514,40.75
1,Contract,4076,1262,30.96
2,Transient-Party,25124,6389,25.43
3,Group,577,59,10.23
